In [1]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [2]:
import pyspark
from pyspark.sql import *
from pyspark.sql import Row
from pyspark.sql.types import *
import re
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql import functions as f
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.functions import col,split
from pyspark.ml.feature import StopWordsRemover,Tokenizer,CountVectorizer
from pyspark.ml.feature import HashingTF,IDF,IDFModel

In [3]:
from pyspark.mllib.classification import SVMModel,SVMWithSGD
from pyspark.mllib.classification import LogisticRegressionWithSGD,SVMWithSGD
from pyspark.mllib.tree import RandomForest
from pyspark.ml.classification import LogisticRegression,LogisticRegressionModel

In [4]:
sc=pyspark.SparkContext()

In [4]:
MAX_MEMORY='8g'

In [5]:
spark= SparkSession \
       .builder \
       .appName("P") \
       .config("spark.some.config.option","some-value") \
       .config("spark.executor.memory",MAX_MEMORY)\
       .config("spark.driver.memory",MAX_MEMORY)\
       .getOrCreate() 

In [6]:
train = pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
m=pd.read_csv('mapping.csv')

In [7]:
tdata=spark.createDataFrame(train)
tedata=spark.createDataFrame(test)
mdata=spark.createDataFrame(m,['id','genre'])

In [8]:
tdata=tdata.withColumn("plot",f.regexp_replace(f.col("plot"),r'[^A-Za-z0-9 ]',''))

In [9]:
rt = RegexTokenizer(inputCol = "plot", outputCol = "o1", pattern = "\\W" )
tdata=rt.transform(tdata)
remover = StopWordsRemover(inputCol = "o1" ,outputCol="so2")
tdata=remover.transform(tdata)
numfeatures=32
htf=HashingTF(inputCol="so2",outputCol="rfeatures",numFeatures=382)
tdata=htf.transform(tdata)
idf=IDF(inputCol="rfeatures",outputCol="features")
idfmodel=idf.fit(tdata)
tdata1=idfmodel.transform(tdata)

In [10]:
#tdata1.select("features").show()
#for i in tdata1.collect():
 #   print(type(i))
m1=mdata.select("genre","id").rdd.collectAsMap()

In [11]:
y=tdata.select('genre')
m1=mdata.select("genre","id").rdd.collectAsMap()
z=y.withColumn('genre',regexp_replace("genre",r'[^A-za-z ,/-]',""))

x=[]
for r in z.rdd.collect(): 
    g=r['genre']
    g=re.sub(r'[^A-Za-z ,/-]',"",g)
    g=list(g.split(","))
    yo=[]
    for c in g:
        c=c.strip()
        yo.append(m1[c])
    iz=[]
    for izx in range(0,len(m1)): 
        iz.append(0)
    #print(iz)
    for iz1 in yo:
        #print(iz1)
        iz[iz1]=1
        
    x.append(iz)
yu=spark.createDataFrame(x,ArrayType(IntegerType()))
#print(type(tdata))
#print(type(yu))

In [12]:
tdata1=tdata1.withColumn("row_id",monotonically_increasing_id())
yu=yu.withColumn("row_id",monotonically_increasing_id())
r=tdata1.join(yu,on=["row_id"],how='inner').drop("row_id")


In [13]:
yc=r.select("movie_id","features","value")

In [14]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

In [15]:
testdata=pd.read_csv('test.csv')
test1=spark.createDataFrame(testdata)
test1=rt.transform(test1)
test1=remover.transform(test1)
test1=htf.transform(test1)
test1=idfmodel.transform(test1)

In [16]:

from pyspark.ml.feature import VectorAssembler
x=[]
for z in range(0,len(m1)):
    p = yc.rdd.map(lambda xu:LabeledPoint(xu.value[z],
                                          Vectors.fromML(xu.features)))
    print("created labeled points")
    lr = LogisticRegressionWithSGD.train(p,iterations=10)
    print("training done")
    for zi in test1.collect():
        x1=zi['movie_id']
        y=lr.predict(Vectors.fromML(zi['features']))
        pt=[str(x1),str(y)]
        x.append(pt)
    print("testing done")

created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done
created labeled points
training done
testing done


In [17]:
presults={}
count=0
for jx in x:
        jx[1]=str(int(float(jx[1])))
        if jx[0] in presults.keys():
            z=presults[jx[0]]
            z.append(jx[1])
        else:
            z1=[]
            z1.append(jx[1])
            presults[jx[0]]=z1
#print(presults)

In [18]:
import csv
for i in range(0,2):
    a_file=open("results2.csv","w")
    writer =csv.writer(a_file)
    writer.writerow(["movie_id","predictions"])
    for k,v in presults.items():
        x2=""
        for v1 in v:
            x2=x2+" "+str(v1)
            x2=x2.strip()
        writer.writerow([k,x2])